In [ ]:
!pip install -q llama-index==0.12.12 huggingface-hub==0.30 instructor==1.8.3 ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.1/481.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
sources_to_process = [
    'azure-architecture',
    'azure-updates',
    'github-samples', 
    'microsoft-learn',
    #'tech-community' # too big & expensive

]

In [ ]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.
import nest_asyncio
nest_asyncio.apply()

#%pdb on


In [ ]:
# set variables
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN2')
HF_TOKEN = userdata.get('HF_TOKEN2')


In [ ]:
# prompt: download all jsonl files from hugging face

from huggingface_hub import HfApi, snapshot_download

# Set the repository ID
repo_id = "vicpada/AzureResources"  # Replace with the actual repo ID

# Define the allow_patterns to only download jsonl files
allow_patterns = "*.jsonl"

# Download the files
snapshot_download(
    repo_id=repo_id,
    repo_type="dataset",
    allow_patterns=allow_patterns,
    local_dir="./downloaded_jsonl_files",  # Specify a local directory to save the files
    token=HF_TOKEN,
)

print("JSONL files downloaded successfully!")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

azure-updates.jsonl:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

tech-community.jsonl:   0%|          | 0.00/2.14G [00:00<?, ?B/s]

github-samples.jsonl:   0%|          | 0.00/172M [00:00<?, ?B/s]

microsoft-learn.jsonl:   0%|          | 0.00/270M [00:00<?, ?B/s]

azure-architecture.jsonl:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

JSONL files downloaded successfully!


In [ ]:
from llama_index.core.schema import Document

def create_docs(input_file: str) -> list[Document]:
    with open(input_file, "r") as f:
        documents: list[Document] = []
        for line in f:
            data = json.loads(line)
            documents.append(
                Document(
                    doc_id=data["doc_id"],
                    text=data["content"],
                    metadata={  # type: ignore
                        "url": data["url"],
                        "title": data["name"],
                        "tokens": data["tokens"],
                        "retrieve_doc": data["retrieve_doc"],
                        "source": data["source"],
                    },
                    excluded_llm_metadata_keys=[
                        "title",
                        "tokens",
                        "retrieve_doc",
                        "source",
                        "context_added",
                    ],
                    excluded_embed_metadata_keys=[
                        "url",
                        "tokens",
                        "retrieve_doc",
                        "source",
                        "context_added",
                    ],
                )
            )
    return documents

In [ ]:
# prompt: foreach jsonl file create a doc

import os
import json
from typing import List

def remove_extension(filename):
    return os.path.splitext(filename)[0]

# Ensure llama_index and huggingface_hub are installed
# !pip install -q llama-index==0.12.12 huggingface-hub==0.30 # Already in preceding code

# Assuming the files are downloaded to "./downloaded_jsonl_files" as per the preceding code
download_dir = "./downloaded_jsonl_files"

all_docs : dict[str,List[Document]] = {}
for filename in os.listdir(download_dir):
    if filename.endswith(".jsonl"):
        source_name = remove_extension(filename)
        if (source_name in sources_to_process):
          filepath = os.path.join(download_dir, filename)
          print(f"Processing file: {filepath}")
          docs_from_file = create_docs(filepath)
          source_name = remove_extension(filename)
          all_docs[source_name]=docs_from_file


print(f"Created {len(all_docs)} documents from all JSONL files.")
# You can now work with the `all_docs` list

Processing file: ./downloaded_jsonl_files/tech-community.jsonl
Created 1 documents from all JSONL files.


In [ ]:
from openai import AsyncOpenAI
import instructor

aclient = AsyncOpenAI()
client: instructor.AsyncInstructor = instructor.from_openai(aclient)

In [ ]:
#from huggingface_hub import InferenceClient

#hf_client = InferenceClient(
#    provider="hf-inference",
#    api_key=os.environ["HF_TOKEN"],
#)

In [ ]:
# prompt: write a function my_before_sleep that writes into a local file instead of printing, and takes into account that it will be called concurrently by many threads

import threading
from tenacity import RetryCallState

# Create a lock for thread-safe file writing
#file_lock = threading.Lock()
log_file_path = "my_before_sleep_log.txt"

def my_before_sleep(retry_state: RetryCallState):
    """
    Writes a message to a local file in a thread-safe manner.

    Args:
        message (str): The message to write.
    """
    message = f"count: {retry_state.attempt_number}, seconds: {retry_state.seconds_since_start}, slept:{retry_state.idle_for}"
    with open(log_file_path, "a") as f:
        f.write(message + "\n")




In [ ]:
from tenacity import retry, stop_after_attempt, wait_exponential, wait_random_exponential, before_sleep_log
from jinja2 import Template
from pydantic import BaseModel, Field
import logging
import sys
from typing import List, Tuple # Import Tuple here
import ipdb



#def my_before_sleep(retry_state: RetryCallState):
#  print(f"count: {retry_state.attempt_number}, seconds: {retry_state.seconds_since_start}, slept:{retry_state.idle_for}")


# Define a Pydantic model for the tuple item
class ContextItem(BaseModel):
    chunk_id: str = Field(..., description="The ID of the chunk.")
    context_text: str = Field(..., description="The context text for the chunk.")

class SituatedContext(BaseModel):
    title: str = Field(..., description="The title of the document.")
    # Use a List of the new ContextItem model
    context: List[ContextItem] = Field(
        ..., description="list of chunk id and the context to situate the chunk within the document."
    )

@retry(stop=stop_after_attempt(4), wait=wait_exponential(multiplier=1, min=4, max=10),  before_sleep=my_before_sleep)
#@retry(wait=wait_random_exponential(min=1, max=180), stop=stop_after_attempt(6), before_sleep=my_before_sleep)
async def situate_context(doc: str, chunks: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
    template = Template(
        """
<document>
{{ doc }}
</document>

Here is the chunks we want to situate within the whole document above:

<chunks>
{% for chunk in chunks %}

<chunk_id>
{{ chunk[0] }}
</chunk_id>

<chunk_text>
{{ chunk[1] }}
</chunk_text>

{% endfor %}
</chunks>



For each chunk, give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
Answer only with a list of chunk id and the succinct context and nothing else.
"""
    )

    #ipdb.set_trace()
    content = template.render(doc=doc, chunks=chunks)

    response = await client.chat.completions.create(
        model="gpt-4.1-nano",
        #max_tokens=1000,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        response_model=SituatedContext,
    )

#    response = await hf_client.chat.completions.create(
#        model="mistralai/Mistral-7B-Instruct-v0.3",
#        max_tokens=500,
#        temperature=0,
#        messages=[
#            {
#                "role": "user",
#                "content": content,
#            }
#        ],
#        response_model=SituatedContext,
#    )

    #return response.context

    # Convert the response back to the original List[Tuple[str, str]] format if needed by downstream code
    # Based on the traceback, the error is in the API call itself due to schema, not the return type handling.
    # However, the calling function `process_chunk` expects `List[Tuple[str, str]]`
    # So, we need to convert the response back to the expected format.
    return [(item.chunk_id, item.context_text) for item in response.context]



In [ ]:
from llama_index.core.schema import TextNode
from instructor.exceptions import InstructorRetryException, IncompleteOutputException
from tenacity import RetryError
import tiktoken
import csv


def save_file(lines: List[Tuple[str,str]], filename):
  with open(filename, "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(lines)

async def process_chunk(nodes: List[TextNode], doc: Document):
    #doc_id: str = node.source_node.node_id  # type: ignore
    #doc: Document = document_dict[doc_id]
    doc_id: str = doc.doc_id
    context_key = "context_added"

    def node_processed(node: TextNode) -> bool:
        return node.metadata.get(context_key, False)

    # Corrected: Create a list of tuples (node_id, node_text)
    filtered_nodes: dict[str, TextNode] = {node.node_id: node for node in nodes if node.source_node and node.source_node.node_id == doc_id and not node_processed(node)}
    if not filtered_nodes:
        return
    filtered_chunks: List[Tuple[str,str]] = [(node_id, filtered_nodes[node_id].text) for node_id in filtered_nodes]


    context_processed = doc.metadata.get(context_key, False)
    if context_processed:
        return

    if doc.metadata["tokens"] > 90_000:
        # Tokenize the document text
        encoding = tiktoken.get_encoding("cl100k_base")
        tokens = encoding.encode(doc.get_content())

        # Trim to 120,000 tokens
        trimmed_tokens = tokens[:90_000]

        # Decode back to text
        trimmed_text = encoding.decode(trimmed_tokens)

        # Update the document with trimmed text
        doc = Document(text=trimmed_text, metadata=doc.metadata)
        doc.metadata["tokens"] = 90_000

    try:
      chunk_contexts: List[Tuple[str,str]] = await situate_context(doc.get_content(), filtered_chunks)
      # save_file(chunk_contexts, f"{doc_id}.txt")
      for node_id, context in chunk_contexts:
        if node_id in filtered_nodes: # Add a check in case API returns unexpected node_ids
          node = filtered_nodes[node_id]
          node.text = f"{node.text}\n\n{context}"
          node.metadata[context_key] = True
        else:
          print(f"Warning: Received context for unknown node_id: {node_id}")
      doc.metadata[context_key] = True
    except InstructorRetryException as e:
        print(f"Failed to process document {doc_id} after multiple retries: {e}")
        pass
    except RetryError as e:
        print(f"Retry Failed to process document {doc_id} after multiple retries: {e}")
        pass




In [ ]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter

def get_all_nodes(
    documents: List[Document]
) -> List[TextNode]:
    # From the document, we create chunks
    pipeline = IngestionPipeline(
        transformations=[SentenceSplitter(chunk_size=800, chunk_overlap=0)]
    )
    all_nodes: list[TextNode] = pipeline.run(documents=documents, show_progress=True)
    print(f"Number of nodes: {len(all_nodes)}")

    return all_nodes


In [ ]:
# create nodes
all_nodes={}
for source_name in all_docs:
  if (source_name not in sources_to_process): continue
  nodes = get_all_nodes(all_docs[source_name])
  all_nodes[source_name] = nodes
  print(f"Number of nodes for {source_name}: {len(nodes)}")


Parsing nodes:   0%|          | 0/14856 [00:00<?, ?it/s]

Number of nodes: 745636
Number of nodes for tech-community: 745636


In [ ]:
# prompt: process nodes in batch of 100 with a minute delay
import pickle
from llama_index.core import Document
import asyncio
from tqdm.asyncio import tqdm

async def process(
    documents: List[Document],
    nodes: List[TextNode],
    batch_size: int = 100, # Added batch_size parameter
    semaphore_limit: int = 50,
    delay: int = 60 # Changed default delay to 60 seconds (1 minute)
):

    semaphore = asyncio.Semaphore(semaphore_limit)

    async def process_with_semaphore(doc, nodes):
        async with semaphore:
          await process_chunk(nodes, doc)
          await asyncio.sleep(0.5)

    # Group documents into batches
    document_batches = [documents[i:i + batch_size] for i in range(0, len(documents), batch_size)]

    for i, batch in enumerate(document_batches):
        print(f"Processing batch {i+1}/{len(document_batches)}")
        tasks = [process_with_semaphore(doc, nodes) for doc in batch]
        await tqdm.gather(*tasks, desc=f"Processing batch {i+1}")
        if i < len(document_batches) - 1:
            print(f"Batch {i+1} completed. Waiting for 1 minute before processing next batch...")
            await asyncio.sleep(delay) # Wait half minute between batches

# Enhance docs

semaphore_limit = 50
delay = 20 # This delay is between batches
dataset_repo_id = "vicpada/AzureResources" # Define dataset_repo_id
batch_size = 7500 # Define batch size

def save_pickle(obj, filename):
  with open(filename, "wb") as f:
      pickle.dump(obj, f)

def load_pickle(filename):
  with open(filename, "rb") as f:
      return pickle.load(f)


def upload_to_hf(filename, repo_id,token):
  api = HfApi(token = token)
  api.upload_file(
  path_or_fileobj=filename,
  path_in_repo=filename,
  repo_id=repo_id,
  repo_type="dataset",
  token=token,
)

for source_name in all_nodes:
  if (source_name not in sources_to_process): continue
  print(f"Processing {source_name}")
  await process(all_docs[source_name], all_nodes[source_name], batch_size=batch_size, semaphore_limit=semaphore_limit, delay=delay)
  enhanced_nodes_for_source = all_nodes[source_name] # This variable name is clearer
  save_pickle(enhanced_nodes_for_source, f"enhanced_nodes_{source_name}.pkl")
  upload_to_hf(f"enhanced_nodes_{source_name}.pkl", dataset_repo_id, HF_TOKEN)


Processing tech-community
Processing batch 1/2


Processing batch 1:   0%|          | 0/7500 [00:00<?, ?it/s]

Processing batch 1:   0%|          | 1/7500 [00:59<124:16:56, 59.66s/it]

Processing batch 1:   0%|          | 2/7500 [01:01<52:58:22, 25.43s/it] 

Processing batch 1:   0%|          | 3/7500 [01:06<34:00:00, 16.33s/it]

...

Processing batch 1:  29%|██▊       | 2141/7500 [1:37:26<15:36:18, 10.48s/it]

Retry Failed to process document ce54b900-cf9b-578b-b826-68d4ecfb1979 after multiple retries: RetryError[<Future at 0x7837ee0eb250 state=finished raised InstructorRetryException>]


Processing batch 1:  29%|██▊       | 2142/7500 [1:37:50<21:56:55, 14.75s/it]

Retry Failed to process document 10eff24d-fade-518b-aac9-cae612411508 after multiple retries: RetryError[<Future at 0x7837ee08eed0 state=finished raised InstructorRetryException>]


Processing batch 1:  29%|██▊       | 2143/7500 [1:37:52<16:18:30, 10.96s/it]